In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import json

In [ ]:
# array of bondlengths used to generate this data
r_array = np.linspace(1.0, 1.8, 50)

# numpy write of the energy eigenvalues at each bondlength
E = np.load("npy_files/CH2O_ccPVDZ_CASCI_88_E_Array.npy")
# numpy write of the dipole moments at each bondlength for each pair of states 
Mu = np.load("npy_files/CH2O_ccPVDZ_CASCI_88_MU_Array.npy")

# commit number of the qed-ci canonical_mo_option branch
sha = '36377939e45c5bcf9d1b6c8d2ecdd6dc29e8ecdd'

# get number of electronic states
N_el = len(E[0,:])

In [ ]:
# equilibrium geometry of formaldehyde - determined from B3LYP/cc-pVDZ
formaldehyde_ccpVDZ_b3lyp_opt = """
0 1
O
C             1    1.203950
H             2    1.120350      1  122.478805
H             2    1.120350      1  122.478805      3  180.000000
symmetry c1
"""

# template for performing CO bond-stretch with all other coordintes frozen at equilibrium values
mol_tmpl = """
0 1
O
C             1    **R**
H             2    1.120350      1  122.478805
H             2    1.120350      1  122.478805      3  180.000000
symmetry c1
"""

# basic psi4 options
options_dict = {
    "basis": "cc-pVDZ",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

In [ ]:
# set up base dictionary - some of this will be updated with each calculation
dictionary = {
  "repo" : {
      
      "repository_url" : " https://github.com/mapol-chem/qed-ci",
      "branch" : "canonical-mo",
      "commit" : sha
  },
    
  "molecule" : {
    "molecule_name": "CH2O",
    "symbols": [
      "O",
      "C", 
      "H",
      "H",
    ],
    "z-matrix" : [],
    "bond_length" : [],
  },
    "driver": "energy",
    "model" : {
        "method" : "casci",
        "orbital_basis"  : options_dict["basis"],
        "active_orbitals" : 8,
        "active_electrons" : 8,
    },
    
    "return_result" : [
        
    ],
    "dipole_data" : "npy_files/CH2O_ccPVDZ_CASCI_88_MU_Array.npy"
}


# function to generate file names based on system details
def generate_file_name(dic):
    
    file_name = dic["molecule"]["molecule_name"] + "_"
    file_name += "CO_Stretch" + "_"
    file_name += str(dictionary["model"]["method"]) + "_"
    file_name += str(dictionary["model"]["orbital_basis"]) + "_"
    file_name += 'n_act_els_' + str(dictionary["model"]["active_electrons"]) + "_"
    file_name += 'n_act_orbs_' + str(dictionary["model"]["active_orbitals"]) + ".json"
    return file_name

In [ ]:
# loop through bondlengths and update relevant dictionary entries
idx = 0
for r in r_array:
    mol_str = mol_tmpl.replace("**R**", str(r))
    dictionary["molecule"]["z-matrix"].append(mol_str)
    dictionary["molecule"]["bond_length"].append(r)
    
    # going to loop over all N_el states and capture all electronic energy eigenvalues
    en = []
    for i in range(N_el):
        en.append(E[idx, i])
    dictionary["return_result"].append(en)
    idx += 1
    

    

In [ ]:
# write to json file
file_name = generate_file_name(dictionary)
json_object = json.dumps(dictionary, indent=4)
with open(file_name, "w") as outfile:
    outfile.write(json_object)